# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt_tab')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer

import pickle


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hambekal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Pandas  impored for data manipulation and sqlalchemy for connection to SQLite database.
After a connection is created to the SQLite database ,data is loaded into a DataFrame.
X is the feature variable containing the messages and Y is the target variables containing the categories.

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df  = pd.read_sql_table('Messages', engine)
#df.head()
X = df['message']  # Assuming 'message' is the feature column
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)  # Drop non-target columns(original has 16046 nulls)

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Todo: tokenize text
    tokens = word_tokenize(text)
    
    # Todo: lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [4]:
#Testing the Tokenization function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize ,token_pattern=None)),  # Convert text data to TF-IDF features using custom tokenizer(Focuses on the importance of words by considering their frequency and their overall presence in the dataset.)
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classifier with RandomForest
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

 # train classifier(pipeline)
pipeline.fit(X_train, y_train)



Pipeline(steps=[('tfidf',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x00000269D44AC900>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)

# Convert predictions to DataFrame for easier handling
predictions_df = pd.DataFrame(y_pred, columns=y_test.columns)

# Iterate through each category and print the classification report
for column in y_test.columns:
    print(f"Category: {column}")
    print(classification_report(y_test[column], predictions_df[column]))




Category: related
              precision    recall  f1-score   support

           0       0.69      0.42      0.52      1492
           1       0.84      0.94      0.89      5009
           2       0.85      0.21      0.33        53

    accuracy                           0.82      6554
   macro avg       0.79      0.52      0.58      6554
weighted avg       0.80      0.82      0.80      6554

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5425
           1       0.84      0.47      0.61      1129

    accuracy                           0.89      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.5

c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'tfidf', 'clf', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__monotonic_cst', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estim

In [15]:
 # specifing parameters for grid search
parameters = {
    'clf__estimator__n_estimators': [100, 200],
    #'clf__estimator__max_depth': [None, 10, 20, 30],
    'clf__estimator__min_samples_split': [2, 5]
}



cv = GridSearchCV(pipeline, param_grid=parameters ,cv=5,verbose=2 )
                
#cv.fit(X_train, y_train)


In [16]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 7.5min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 8.0min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 7.7min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 8.8min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 8.4min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time=18.0min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time=19.7min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time=27.9min
[CV] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time=23.9min
[CV] END clf__estimator__min_samples_split=2, clf

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x00000269D44AC900>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5],
                         'clf__estimator__n_estimators': [100, 200]},
             verbose=2)

In [17]:
cv.cv_results_

{'mean_fit_time': array([ 471.64455261, 1594.65272384,  393.69925089,  811.45247388]),
 'std_fit_time': array([ 28.39354208, 598.15483297,  17.06888671,  85.09668558]),
 'mean_score_time': array([12.00386734, 27.5378315 , 11.22018485, 24.95404468]),
 'std_score_time': array([1.5870944 , 5.42232678, 0.72141818, 2.78840039]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 5, 5],
              mask=[False, False, False, False],
        fill_value=999999),
 'param_clf__estimator__n_estimators': masked_array(data=[100, 200, 100, 200],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 200},
  {'clf__estimator__min_samples_split': 5,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 5,
   'clf__estimator__n_estimators': 200}],
 'split0_test_scor

In [ ]:
#Best Model parameters
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}


In [19]:
print(cv.best_estimator_)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x00000269D44AC900>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])


In [22]:
#building new model
best_model = cv.best_estimator_
print(best_model)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x00000269D44AC900>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
y_pred = best_model.predict(X_test)

for i in range(36):
    print("=============================",y_test.columns[i], '=================================')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

============================= related =================================
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1492
           1       0.84      0.95      0.89      5009
           2       0.85      0.21      0.33        53

    accuracy                           0.82      6554
   macro avg       0.80      0.52      0.58      6554
weighted avg       0.81      0.82      0.80      6554

============================= request =================================
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5425
           1       0.84      0.50      0.63      1129

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

============================= offer =================================
              precision    recall  f1-score   support

           0       1.00

c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6517
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

============================= hospitals =================================
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482
           1       0.00      0.00      0.00        72

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

============================= shops =================================
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                          

c:\Users\hambekal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 9. Export your model as a pickle file

In [27]:
# Save the model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.